<a href="https://colab.research.google.com/github/NirantK/Hinglish/blob/cleanlab/CleanlabDistilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What is cleanlab?

<br /> 

[cleanlab](https://l7.curtisnorthcutt.com/cleanlab-python-package) is a machine learning python package for learning with noisy labels and finding label errors in datasets. cleanlab CLEANs LABels. It is powered by the theory of confident learning

In [ ]:
# !pip install cleanlab

In [ ]:
from pathlib import Path
import pandas as pd
import sys
sys.path.insert(0, "../")
from hinglishutils import get_files_from_gdrive

In [ ]:
from pathlib import Path
datapath = Path("../data")
data_raw = datapath/"raw"
data_interim = datapath/"interim"
data_processed = datapath/"processed"
cleanlab_datapath = datapath/"cleanlab"

In [ ]:
url = (
    "https://drive.google.com/file/d/1T6N_ba6-w2xLM6t4_EINGR7SGVQpCXd_/view?usp=sharing"
)

get_files_from_gdrive(url, str(data_raw/"distilBertOutput.csv"))
DistilBert = pd.read_csv(data_raw/"distilBertOutput.csv")
DistilBert

In [ ]:
vals = {"neutral": 0, "negative": 1, "positive": 2}
DistilBert["labels"] = DistilBert.Sentiment.map(vals)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(DistilBert["clean_text"])
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

tfidf_transformer = TfidfTransformer()
sents = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
DistilBert

In [ ]:
import cleanlab

`py_train` -> What is the percentage of each labels in the dataset

In [ ]:
py_train = cleanlab.util.value_counts(DistilBert["labels"]) / float(
    len(DistilBert["labels"])
)
py_train

In [ ]:
prune_method = "prune_by_noise_rate"

In [ ]:
import numpy as np

np.array(list(DistilBert["labels"]))

In [ ]:
from sklearn.linear_model import LogisticRegression as LogReg

(jc, psx) = cleanlab.latent_estimation.estimate_confident_joint_and_cv_pred_proba(
    sents,
    np.array(list(DistilBert["labels"])),
    clf=LogReg(multi_class="auto", solver="lbfgs", max_iter=10000, verbose=True),
)

In [ ]:
est_py, est_nm, est_inv = cleanlab.latent_estimation.estimate_latent(
    jc, np.array(list(DistilBert["labels"]))
)

In [ ]:
ordered = cleanlab.pruning.get_noise_indices(
    np.array(list(DistilBert["labels"])),
    psx,
    #    est_inv,
    prune_method=prune_method,
    frac_noise=0.37,
    sorted_index_method="normalized_margin",
)
print("Number of estimated errors in test set:", len(ordered))

In [ ]:
len(ordered)

In [ ]:
vals = {0: "negative", 1: "neutral", 2: "positive"}
errors = []
for idx in ordered[:100]:
    errors.append(
        f"Estimated Error in {idx}\npredicted {DistilBert['Sentiment'][idx]}\n sentence {DistilBert['clean_text'][idx]}\n -----"
    )

In [ ]:
with open(cleanlab_datapath/'errorsDistilBert.txt', 'w') as f: 
    for item in errors:
        f.write("%s\n" % item)